In [1]:
import pandas as pd
from src.seq2seq.dataset import SeqDataset, pad_batch
from src.seq2seq.metrics import compute_metrics
from torch.utils.data import DataLoader

DATA_PATH = 'data/ArchiveII/ArchiveII_seq_sim_pp_motifs/train_0.csv'
df = pd.read_csv(DATA_PATH)

display(df.sample(2))
data = SeqDataset(dataset_path=DATA_PATH)
# print(data[0])
for k, i in data[0].items():
    print(f'key: {k},   type:{type(i)}')

,Unnamed: 0,id,fam,sequence,len,structure,base_pairs,pseudo_probe,motifs,stem
816,5015,srp_Mann.succ._AE016827,srp,GGGGGCUCUGUUGGUUCUCCCGCAACGCUACUCUGUUUACCAGGUC...,118,((((((((((((((((.((((((.((((...((((((.....(((....,"[[1, 110], [2, 109], [3, 108], [4, 107], [5, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",SSSSSSSSSSSSSSSSISSSSSSBSSSSIIISSSSSSIIIIISSSI...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
303,1875,5s_Dactylosporangium-aurantiacum-1,5s,GUUACGGCGGCCAUAGCGGAGGGGAAACGCCCGGUCCCAUUCCGAA...,121,(((.((((((.....((((((((.....((((((...............,"[[1, 119], [2, 118], [3, 117], [5, 116], [6, 1...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",SSSBSSSSSSMMMMMSSSSSSSSIIIIISSSSSSHHHHHHHHHHHH...,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."


key: id,   type:<class 'str'>
key: length,   type:<class 'int'>
key: sequence,   type:<class 'str'>
key: embedding,   type:<class 'torch.Tensor'>
key: pseudo_probing,   type:<class 'torch.Tensor'>
key: stem,   type:<class 'torch.Tensor'>
key: motif_emb,   type:<class 'torch.Tensor'>


In [2]:
from functools import partial
pad_batch_with_fixed_length = partial(pad_batch, fixed_length=128)
batch_size = 1
loader = DataLoader(
    SeqDataset(DATA_PATH),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    collate_fn=pad_batch_with_fixed_length,
)
load = iter(loader)
d = next(load)
d2 = next(load)

In [3]:
d['length']
d2['length']

[75]

In [4]:
compute_metrics(d['motif_emb'],d2['motif_emb'], d2['mask'], binary=False)

{'F1': 0.4000000000000001, 'Accuracy': 0.4, 'Accuracy_seq': 0.0}

In [5]:
compute_metrics(d['pseudo_probing'],d2['pseudo_probing'], d2['mask'], binary=True)

{'F1': 0.611764705882353, 'Accuracy': 0.56, 'Accuracy_seq': 0.0}

In [6]:
import torch as tr
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
def metrics_test(x,y,mask, expected):
    out = compute_metrics(x, y, mask, binary=True)
    acc = accuracy_score(x.reshape(6).numpy(), y.reshape(6).numpy())
    f1 = f1_score(x.reshape(6).numpy(), y.reshape(6).numpy())
    print(f'Expected: {expected}\n')
    print(f'output: {out}\n')
    print(f'sklearn: f1: {f1},  acc: {acc},\n')

In [9]:
import torch as tr
import numpy as np
from sklearn.metrics import accuracy_score

x = tr.Tensor([[[ 0.,  1.,  0.], ], 
               [[ 1.,  1.,  1.], ], 
               ])

y = tr.Tensor([[[ 0.,  1.,  0.], ], 
               [[ 1.,  0.,  1.],], 
               ])
mask = tr.zeros(2, 3, dtype= tr.bool)
mask[:,:] = True
print(x.shape)
print(y.shape)
print(mask.shape)
metrics_test(x,y,mask,expected={'acc':0}) 

torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 3])
Expected: {'acc': 0}

output: {'F1': 0.8571428571428571, 'Accuracy': 0.8333333333333334, 'Accuracy_seq': 0.5}

sklearn: f1: 0.8571428571428571,  acc: 0.8333333333333334,



In [67]:
print(x.reshape(6))
print(y.reshape(6))

tensor([0., 1., 0., 1., 1., 1.])
tensor([0., 1., 0., 1., 0., 1.])


In [ ]:

TP = ((x == 1) & (y == 1)).sum().item()
FP = ((x == 1) & (y == 0)).sum().item()
FN = ((x == 0) & (y == 1)).sum().item()
TN = ((x == 0) & (y == 0)).sum().item()
total = mask.sum().item()
accuracy = TP/total if total>0 else 0.0
precision = TP/(TP+FP) if (TP+FP)>0 else 0.0
recall    = TP/(TP+FN) if (TP+FN)>0 else 0.0
f1        = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0.0

print(
    f'''
    TP       : {TP}
    FP       : {FP}
    FN       : {FN}
    TN       : {TN}
    accuracy : {accuracy}       
    precision: {precision}
    recall   : {recall}   
    f1       : {f1}
    '''
)


    TP       : 3
    FP       : 1
    FN       : 0
    TN       : 2
    accuracy : 0.8333333333333334       
    precision: 0.75
    recall   : 1.0   
    f1       : 0.8571428571428571
    


In [ ]:
x.argmax(dim=1)

tensor([[0],
        [0]])

In [ ]:
y.argmax(dim=1)

tensor([[0],
        [0]])